[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# SQLite3: La Base de Datos de la Biblioteca Estándar

Python incluye `sqlite3`, un motor de base de datos SQL relacional, ligero y sin servidor. A diferencia de PostgreSQL o MySQL, SQLite no tiene un proceso separado; el motor es parte de tu programa y lee/escribe directamente en un archivo.

## ¿Cuándo usar SQLite?
✅ Prototipos rápidos.
✅ Aplicaciones de escritorio o móviles (es el estándar en Android/iOS).
✅ Archivos de configuración complejos o caché local.
❌ Aplicaciones web con miles de escrituras concurrentes.

In [ ]:
import sqlite3
import os

# Usaremos una base de datos en memoria para este ejemplo
# Para usar un archivo: sqlite3.connect('mi_base.db')
conn = sqlite3.connect(':memory:')

# El cursor es nuestro intermediario para ejecutar comandos SQL
cursor = conn.cursor()

print(f"Conectado a SQLite versión: {sqlite3.sqlite_version}")

## 3. Data Definition Language (DDL)
Creando tablas y esquemas.

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS usuarios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    email TEXT UNIQUE,
    edad INTEGER
)
""")
print("Tabla 'usuarios' creada.")

## 4. Manipulación de Datos (CRUD)

### Insertar Datos
⚠️ **IMPORTANTE**: Siempre usa marcadores de posición (`?`) para evitar inyección SQL. Nunca concatenes cadenas.

In [ ]:
# Insertar un solo registro
cursor.execute("INSERT INTO usuarios (nombre, email, edad) VALUES (?, ?, ?)", 
               ("Alice", "alice@example.com", 30))

# Insertar múltiples registros
usuarios_nuevos = [
    ("Bob", "bob@example.com", 25),
    ("Charlie", "charlie@example.com", 35),
    ("Diana", "diana@example.com", 28)
]
cursor.executemany("INSERT INTO usuarios (nombre, email, edad) VALUES (?, ?, ?)", 
                   usuarios_nuevos)

# Guardar cambios
conn.commit()
print(f"Registros insertados totales: {cursor.rowcount}")

### Consultar Datos (SELECT)

In [ ]:
# Consultar todos
print("--- Todos los usuarios ---")
for fila in cursor.execute("SELECT * FROM usuarios"):
    print(fila)

# Consultar con filtro
print("\n--- Usuarios mayores de 29 ---")
cursor.execute("SELECT nombre, edad FROM usuarios WHERE edad > ?", (29,))
resultado = cursor.fetchall()
print(resultado)

## 5. Transacciones y Context Managers

Podemos usar la conexión como un Context Manager. Si el bloque termina bien, hace `commit`. Si hay una excepción, hace `rollback` automáticamente.

In [ ]:
try:
    with conn:
        conn.execute("INSERT INTO usuarios (nombre, email, edad) VALUES (?, ?, ?)", 
                    ("Eve", "eve@example.com", 22))
        
        # Simulamos un error
        # raise Exception("Falló algo crítico")
        
    print("Transacción exitosa")
except Exception as e:
    print(f"Transacción revertida: {e}")

# Verificar
cursor.execute("SELECT nombre FROM usuarios WHERE nombre = 'Eve'")
print(f"¿Eve existe? {cursor.fetchone()}")

conn.close()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2017-2026.</p>